In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re
from selenium.common.exceptions import NoSuchElementException
import csv
import pandas as pd
import os
from urllib.request import urlopen

In [2]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(2)
#상품소개-간편식사-전체 에서 시작
driver.get('https://www.emart24.co.kr/introduce2/findBranch.asp')

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

In [8]:
region_xpath = '//*[@id="stplacesido"]'
option_xpath = '/option[10]'
region = driver.find_element_by_xpath(region_xpath+option_xpath)
region.click()
city_list = [2,6,9,13,16,19,21,22]

total_results = []

for city in city_list:

    city_xpath = '//*[@id="stplacegugun"]'
    city_option = '/option[{}]'.format(city)
    #print(city_xpath+city_option)
    city_find = driver.find_element_by_xpath(city_xpath+city_option)
    driver.implicitly_wait(2)
    city_find.click()
    driver.implicitly_wait(2)
    
    #주소 찾기
    search_xpath = '//*[@id="skipCont"]/div[2]/div[1]/input'
    driver.find_element_by_xpath(search_xpath).click()
    driver.implicitly_wait(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    #skipCont > div.section > div.find_listArea.openList > div > a.next.bgNone
    #page
    cur = 1
    page = soup.select('#skipCont > div.section > div.find_listArea.openList > div > a')
    num = len(page) - 4 #10
    
    for count in range(num):
        #next_button = soup.select('#skipCont > div.section > div.find_listArea.openList > div > a.next.bgNone')
        #print(next_button)
        #next_button.click()
        
        """if cur > num:
            html = driver.page_source
            soup = BeautifulSoup(html, "lxml")
            page = soup.select('#skipCont > div.section > div.find_listArea.openList > div > a')
            num = len(page) - 4 #10"""
        
        next_xpath = '//*[@id="skipCont"]/div[2]/div[2]/div/a[{}]'.format(num+3) #13
        driver.find_element_by_xpath(next_xpath).click()
        driver.implicitly_wait(2)
        cur += 1
        
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        
        for j in range(5):
            try:
                driver.implicitly_wait(5)
                cs_branch = soup.select('#skipCont > div.section > div.find_listArea.openList > table > tbody > tr:nth-of-type({}) > td:nth-of-type(1) > strong'.format(j+1))[0].text
                print(cs_branch)

                cs_info = soup.select('#skipCont > div.section > div.find_listArea.openList > table > tbody > tr:nth-of-type({}) > td.txtLeft > p:nth-of-type(1)'.format(j+1))[0].text
                cs_info_list = cs_info.split(' | ')
                cs_address = cs_info_list[0]
                cs_time = cs_info_list[1]
                cs_time = cs_time.split(' : ')[1]
                print(cs_address)
                print(cs_time)
                attr_names = ['cs_24', 'bean_coffee', 'medicine', 'delivery', 'toto', 'atm','posa', 'lotto', 'cash_withdrawal', 'tax_refund', 'unattended', 'wifi', 'barista', 'reserve']

                attr_array = []
                for index in range(14):
                    attr = soup.select('#skipCont > div.section > div.find_listArea.openList > table > tbody > tr:nth-of-type({}) > td.txtLeft > p.find_listSelect_Img > img:nth-of-type({})'.format(j+1,index+1))[0]
                    #print(attr)
                    if 'off' in str(attr):
                        attr = 0
                    else:
                        attr = 1
                    attr_array.append(attr)  
                print(attr_array)
            except:
                break
            
            result = {}
            result['cs_name'] = '이마트24'
            result['cs_branch'] = cs_branch
            result['cs_address'] = cs_address
            result['cs_time'] = cs_time
            for i in range(len(attr_names)):
                result[str(attr_names[i])] = attr_array[i]
            total_results.append(result)
            
    
    #3933
    

강남오픈로드점
서울특별시 강남구 논현로16길 6 
00:00 ~ 00:00
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
논현101점
서울특별시강남구 선릉로145길 7 (논현동)
00:00 ~ 00:00
[1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1]
논현개나리점
서울특별시강남구논현동 131-23
00:00 ~ 00:00
[1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0]
논현베스틴점
서울특별시 강남구 선릉로111길 40 
00:00 ~ 00:00
[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
논현세기점
서울특별시강남구 학동로20길 33
00:00 ~ 00:00
[1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
논현진성점
서울특별시강남구논현동 176-21
00:00 ~ 00:00
[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
논현청학점
서울특별시 강남구 도산대로 232 이마트24 편의점
08:00 ~ 00:00
[0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
논현행운점
서울특별시강남구 학동로38길 40 (논현동)
06:00 ~ 00:00
[0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
대치은마점
서울특별시 강남구 삼성로58길 39 1층 102~103호
00:00 ~ 00:00
[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
대치타운점
서울특별시 강남구 삼성로85길 34 코너1층
00:00 ~ 00:00
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
대치휘문점
서울특별시강남구대치동 953-15
06:00 ~ 00:00
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
대치SK점
서울특별시 강남구 삼성로51길 11강남구 (대치동)
07:00 ~ 01:00

상도제일점
서울특별시동작구 양녕로28길 28 (상도동)
06:00 ~ 02:00
[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
상도해피점
서울특별시동작구 매봉로 66 (상도1동)
00:00 ~ 00:00
[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
숭실대역점
서울특별시동작구 상도로62길 10
07:00 ~ 01:00
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
신대방역점
서울특별시 동작구 신대방길 5 
00:00 ~ 00:00
[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
신대방점
서울특별시동작구 보라매로19길 25 (신대방동)
07:00 ~ 02:00
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
전문건설회관점
서울특별시동작구신대방동 395-70 , 전문건설회관 지하1층
07:00 ~ 21:00
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
R동작대교노을점
서울특별시 동작구 동작대로 335 서울 동작구 동작동 316-1
07:00 ~ 00:00
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]
전문건설회관점
서울특별시동작구신대방동 395-70 , 전문건설회관 지하1층
07:00 ~ 21:00
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
R동작대교노을점
서울특별시 동작구 동작대로 335 서울 동작구 동작동 316-1
07:00 ~ 00:00
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]
반포가온점
서울특별시서초구 사평대로55길 65-11
00:00 ~ 00:00
[1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
반포서래마을점
서울특별시서초구 서래로5길 96
06:00 ~ 01:00
[0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
반포예

이노플렉스점
서울특별시 영등포구 양산로 57-5 
06:00 ~ 19:00
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
한강성심병원점
서울특별시영등포구 버드나루로7길 7 (영등포동2가)
06:00 ~ 00:00
[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
이노플렉스점
서울특별시 영등포구 양산로 57-5 
06:00 ~ 19:00
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
한강성심병원점
서울특별시영등포구 버드나루로7길 7 (영등포동2가)
06:00 ~ 00:00
[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
용산메가트리움점
서울특별시용산구 한강대로43길 8 (한강로2가 벽산메가트리움)
07:00 ~ 01:00
[0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
용산삼각지점
서울특별시 용산구 한강대로54길 12 
07:00 ~ 00:00
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
용산서계점
서울특별시용산구 청파로85가길 14
08:00 ~ 00:00
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
용산선인상가점
서울특별시용산구 새창로 181 (한강로2가 선인상가)
07:30 ~ 20:00
[0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
용산센트럴점
서울특별시 용산구 한강대로 95용산구한강로2가 B146호
07:00 ~ 01:00
[0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
용산신창점
서울특별시용산구신창동 56-37
06:30 ~ 23:00
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
용산청룡점
서울특별시용산구갈월동 98-38
06:30 ~ 23:00
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
용산트럼프월드점
서울특별시 용산구 

In [5]:
exists = os.path.isfile('./result_emart.csv')

if exists:
    with open('./result_emart.csv', 'a') as csvfile:
        fieldnames = ['cs_name', 'cs_branch', 'cs_address', 'cs_time', 'cs_24', 'bean_coffee', 'medicine', 'delivery', 'toto', 'atm','posa', 'lotto', 'cash_withdrawal', 'tax_refund', 'unattended', 'wifi', 'barista', 'reserve']

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for i in range(len(total_results)):
            writer.writerow(total_results[i])
else:
    with open('./result_emart.csv', 'w') as csvfile:
        fieldnames = ['cs_name', 'cs_branch', 'cs_address', 'cs_time', 'cs_24', 'bean_coffee', 'medicine', 'delivery', 'toto', 'atm','posa', 'lotto', 'cash_withdrawal', 'tax_refund', 'unattended', 'wifi', 'barista', 'reserve']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for i in range(len(total_results)):
            writer.writerow(total_results[i])

In [3]:
import csv
results = []
attr_names = ['cs_24', 'bean_coffee', 'medicine', 'delivery', 'toto', 'atm','posa', 'lotto', 'cash_withdrawal', 'tax_refund', 'unattended', 'wifi', 'barista', 'reserve', 'lat', 'long']

with open('./result_emart_latlong.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        result = {}
        result['cs_name'] = row['cs_name']
        result['cs_branch'] = row['cs_branch']
        result['cs_address'] = row['cs_address']
        result['cs_time'] = row['cs_time']
        for i in range(len(attr_names)):
            result[str(attr_names[i])] = row[str(attr_names[i])]      
        results.append(result)

In [3]:
print(results[112])

NameError: name 'results' is not defined

In [5]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(2)

for result in results[112:]:
    gmapUrl = "https://www.google.com/maps/place/" + result['cs_address']
    driver.get(gmapUrl)
    driver.implicitly_wait(2)
    print(gmapUrl)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    
    while "@" not in driver.current_url:
        driver.implicitly_wait(2)
    if driver.current_url!=gmapUrl:
        print(driver.current_url)
        gmapUrl = driver.current_url
        startIndex = gmapUrl.index("@")
        middleIndex = gmapUrl.index(",")
        lastIndex = gmapUrl.rindex(",")
        lat = gmapUrl[startIndex+1:middleIndex]
        long = gmapUrl[middleIndex+1:lastIndex]

        result['lat'] = lat
        result['long'] = long
        print(gmapUrl)
        print(lat)
        print(long)

https://www.google.com/maps/place/서울특별시서초구 방배동 방배로20길 8-3, 1층 101호


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="searchbox-searchbutton"]"}
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.42.591059 (a3d9684d10d61aa0c45f6723b327283be1ebaad8),platform=Mac OS X 10.12.6 x86_64)


In [12]:
print(results)

[{'cs_name': '이마트24', 'cs_branch': '강남오픈로드점', 'cs_address': '서울특별시 강남구 논현로16길 6\xa0', 'cs_time': '00:00 ~ 00:00', 'cs_24': '0', 'bean_coffee': '0', 'medicine': '0', 'delivery': '0', 'toto': '0', 'atm': '0', 'posa': '0', 'lotto': '0', 'cash_withdrawal': '0', 'tax_refund': '0', 'unattended': '0', 'wifi': '0', 'barista': '0', 'reserve': '0', 'lat': '37.4760144', 'long': '127.0448734'}, {'cs_name': '이마트24', 'cs_branch': '논현101점', 'cs_address': '서울특별시강남구\xa0선릉로145길 7 (논현동)', 'cs_time': '00:00 ~ 00:00', 'cs_24': '1', 'bean_coffee': '1', 'medicine': '1', 'delivery': '1', 'toto': '0', 'atm': '0', 'posa': '1', 'lotto': '0', 'cash_withdrawal': '1', 'tax_refund': '0', 'unattended': '0', 'wifi': '0', 'barista': '1', 'reserve': '1', 'lat': '37.4496641', 'long': '126.8908177'}, {'cs_name': '이마트24', 'cs_branch': '논현개나리점', 'cs_address': '서울특별시강남구논현동\xa0131-23', 'cs_time': '00:00 ~ 00:00', 'cs_24': '1', 'bean_coffee': '1', 'medicine': '1', 'delivery': '0', 'toto': '1', 'atm': '1', 'posa': '1', 'lotto':

In [20]:
with open('./result_emart_latlong.csv', 'w') as csvfile:
    fieldnames = ['cs_name', 'cs_branch', 'cs_address', 'cs_time', 'cs_24', 'bean_coffee', 'medicine', 'delivery', 'toto', 'atm','posa', 'lotto', 'cash_withdrawal', 'tax_refund', 'unattended', 'wifi', 'barista', 'reserve', 'lat', 'long']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(results)):
        writer.writerow(results[i])